In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from operator import itemgetter
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
def MAPE(y_true, y_pred):
    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]
    num = len(y_pred)
    sums = 0
    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp
    mape = sums * (100 / num)
    return mape

import math
def eva_regress(y_true, y_pred):
    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)

In [ ]:
correlations = pd.read_csv('../../DCP-HTO/calculated_correlation/correlation_current.csv')
correlations = correlations.drop('Unnamed: 0',axis=1)

In [ ]:
def one_hot_outliers(df,k):
    intersections = df.columns
    odf = pd.DataFrame()
    for intersection in intersections:
        y = df[intersection]
        boundary = y.sort_values().head(k)[-1]#Get K lowest LOF score and use as boundary 
        print('Boundary is set to: ', boundary)
        odf['ol_'+intersection] = y.apply(make_binary,boundary=boundary)
    return odf
    
#Make the label a binary of outlier or not.
def make_binary(row,boundary):
    if row <= boundary:
        row = 1
    else:
        row = 0
    return row

In [ ]:
tflow_file = '../../DCP-HTO/SFPD-LOF/input_data/other_intersections/'
tflow = pd.read_csv(tflow_file+'K302.csv',index_col=["timestamp"], parse_dates=True)
#sel = ['K173','K414','K158','K402','K305']
sel = [
    'K173','K414','K158','K402','K305',
    'K071','K097','K124','K128','K159','K184','K189','K206','K225','K270','K304','K405','K406','K424','K430','K703','K704','K707','K711','K561','K504','K145','K250',
    'a12in','a12out','n211in','n211out','n141in','n141out','n142in','n142out','n143in','n143out'
]
for i in sel:
    #print(i)
    x = pd.read_csv(tflow_file+i+'.csv',index_col=["timestamp"], parse_dates=True)
    tflow = pd.merge(tflow, x, on='timestamp',how='left')


#for f in filenames:
 #   dataframes.append(pd.read_csv(f))

In [ ]:
tflow = tflow.fillna(method='ffill')
tflow = tflow.fillna(0)

In [ ]:
tflow.shape

In [ ]:
tflow.head()

# All intersection data!

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
# load dataset
#dataset = read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv', header=0,index_col=0)
values = tflow.values
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
# specify the number of lag hours
n_hours = 12
n_features = 41

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 140256
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

In [ ]:
# split into input and outputs
n_obs = n_hours * n_features

In [ ]:
train_X, train_y = train[:, :n_obs], train[:, -n_features]

In [ ]:
test_X, test_y = test[:, :n_obs], test[:, -n_features]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

In [ ]:
# invert scaling for forecast
old_inv_yhat = concatenate((yhat, test_X[:, -40:]), axis=1)
old_inv_yhat = scaler.inverse_transform(old_inv_yhat)
old_inv_yhat = old_inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -40:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, old_inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
eva_regress(inv_y, old_inv_yhat)

In [ ]:
df = {'LSTM':old_inv_yhat[:300],'True data':inv_y[:300]}
df = pd.DataFrame(data=df)
#df = df.set_index('datetime')

In [ ]:
len(inv_y)

# Advanced input data

In [ ]:
#Selection based on dynamic feature reduction
sel=['K302',
 'K173',
 'K414',
 'K158',
 'K402',
 'K305',
 'K128',
 'K184',
 'K206',
 'K225',
 'K304',
 'K406',
 'K704',
 'K711',
 'K561',
 'K504',
 'K145',
 'a12in',
 'a12out',
 'n211out',
 'n141in',
 'n141out',
 'n142in',
 'n143out'
    ]

In [ ]:
fpd = pd.read_csv('../../DCP-HTO/dataframe_creation/complete_dataframe.csv',index_col=0, parse_dates=True)
fpd = fpd.fillna(1)
fpd_sel = fpd[sel]

In [ ]:
odf = one_hot_outliers(fpd_sel,50)

In [ ]:
rodf = odf.resample('15T').pad()

In [ ]:
rodf.shape

In [ ]:
rodf.index.name = 'timestamp'

In [ ]:
tflow_file = '../../DCP-HTO/SFPD-LOF/input_data/other_intersections/'
tflow = pd.read_csv(tflow_file+'K302.csv',index_col=["timestamp"], parse_dates=True)
for i in sel:
    #print(i)
    x = pd.read_csv(tflow_file+i+'.csv',index_col=["timestamp"], parse_dates=True)
    tflow = pd.merge(tflow, x, on='timestamp',how='left')

In [ ]:
tflow = tflow.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'],axis=1)

In [ ]:
tflow = pd.merge(tflow, rodf, on='timestamp',how='left')

In [ ]:
tflow = tflow.fillna(method='ffill')
tflow = tflow.fillna(0)

In [ ]:
dti = tflow['K414']

In [ ]:
dti = dti[140256:]

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
values = tflow.values
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
# specify the number of lag hours
n_hours = 12
n_features =49

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 140256
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

In [ ]:
# split into input and outputs
n_obs = n_hours * n_features

In [ ]:
train_X, train_y = train[:, :n_obs], train[:, -n_features]

In [ ]:
test_X, test_y = test[:, :n_obs], test[:, -n_features]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -48:]), axis=1)

inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -48:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
eva_regress(inv_y, inv_yhat)

In [ ]:
df = {'LSTM':inv_yhat[:300],'True data':inv_y[:300]}
df = pd.DataFrame(data=df)
#df = df.set_index('datetime')

In [ ]:
df.plot(y=['LSTM','True data'],figsize=(20,10),title='Traffic flow outlier predictions')

In [ ]:
ndti = dti[496:]

In [ ]:
ndf = {'LSTM':inv_yhat[484:],'True data':inv_y[484:],'datetime':ndti.index}
ndf = pd.DataFrame(data=ndf)
ndf = ndf.set_index('datetime')

# Outlier zoom

In [ ]:
k302 = pd.read_csv('../dat/traffic_intersection_selection.csv', index_col=["timestamp"], usecols=["timestamp","K414"], parse_dates=True)
testol = k302['2019':]
testolindex = testol.sort_values('K414').head(10).index
strdatetimes = [str(a) for a in testolindex]
sliced = []
for item in strdatetimes:
    sliced.append(item[:13]+':00:00')
    sliced.append(item[:13]+':15:00')
    sliced.append(item[:13]+':30:00')
    sliced.append(item[:13]+':45:00')
test_dat = pd.read_csv('../TFP_adapted/data/new_test.csv',parse_dates=['timestamp'])
olindexes = test_dat[test_dat.timestamp.isin(sliced)].index

In [ ]:
andf = ndf[ndf.index.isin(sliced)]
def get_diff(row):
    return row['True data'] - row ['LSTM']
andf['diff'] = andf.apply(get_diff,axis=1)

In [ ]:
#andf = andf.sort_values('diff',ascending=False)[:10]

In [ ]:
#andf = andf.drop('diff',axis=1)

In [ ]:
eva_regress(andf['True data'], andf['LSTM'])

In [ ]:
andf.plot(kind='bar',figsize=(20,10),title='Advanced traffic flow outlier predictions')

# OLD

In [ ]:
old_ndf = {'LSTM':old_inv_yhat[4068:],'True data':inv_y[484:],'datetime':ndti.index}
old_ndf = pd.DataFrame(data=old_ndf)
old_ndf = old_ndf.set_index('datetime')

In [ ]:
k302 = pd.read_csv('../dat/traffic_intersection_selection.csv', index_col=["timestamp"], usecols=["timestamp","K414"], parse_dates=True)
testol = k302['2019':]
testolindex = testol.sort_values('K414').head(10).index
strdatetimes = [str(a) for a in testolindex]
sliced = []
for item in strdatetimes:
    sliced.append(item[:13]+':00:00')
    sliced.append(item[:13]+':15:00')
    sliced.append(item[:13]+':30:00')
    sliced.append(item[:13]+':45:00')
test_dat = pd.read_csv('../TFP_adapted/data/new_test.csv',parse_dates=['timestamp'])
olindexes = test_dat[test_dat.timestamp.isin(sliced)].index

In [ ]:
bandf = old_ndf[old_ndf.index.isin(sliced)]
def get_diff(row):
    return row['True data'] - row ['LSTM']
bandf['diff'] = bandf.apply(get_diff,axis=1)

In [ ]:
#bandf = bandf.sort_values('diff',ascending=False)[:10]

In [ ]:
old_diff = bandf['diff']

In [ ]:
bandf = bandf.drop('diff',axis=1)
bandf = bandf.sort_index()

In [ ]:
eva_regress(bandf['True data'], bandf['LSTM'])

In [ ]:
bandf.plot(kind='bar',figsize=(20,10),title='Naive traffic flow outlier predictions')

In [ ]:
import matplotlib as mpl
def plot_results(y_true, y_preds, names):
    """Plot
    Plot the true data and predicted data.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
        names: List, Method names.
    """
    d = '2020-1-4 00:00'
    x = pd.date_range(d, periods=len(y_true), freq='15min')

    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.plot(x, y_preds[0], label=names[0],color='#ff7f0e')
    ax.plot(x, y_true, label='True Data',color='#1f77b4')
        

    plt.legend()
    plt.grid(True)
    plt.xlabel('Time of Day')
    plt.ylabel('Flow')

    date_format = mpl.dates.DateFormatter("%d-%m %H:%M")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

    plt.show()

In [ ]:
plot_pred = [old_ndf['LSTM'][300:600]]
plot_true = old_ndf['True data'][300:600]
plot_results(plot_true,plot_pred,['Multivariate LSTM'])

# Combined plot

In [ ]:
naive_pred = bandf.LSTM

In [ ]:
andf['naive'] = naive_pred
andf['old_diff'] = old_diff

In [ ]:
andf[['LSTM','True data','naive']].plot(kind='bar',figsize=(20,10),title='Naive traffic flow outlier predictions')

# Hourly

In [ ]:
nandf = andf.resample('H').first()
nandf = nandf[nandf.LSTM >0]
#nandf = nandf.drop('diff',axis=1)

In [ ]:
del nandf.index.name

In [ ]:
nandf = nandf.rename(columns={"naive": "Multivariate LSTM"})
nandf = nandf.rename(columns={"LSTM": "OE-LSTM"})

In [ ]:
temp_ind = nandf.index

In [ ]:
new_ind = []
for item in temp_ind:
    new_ind.append(str(item)[5:-3])

In [ ]:
nandf.index = new_ind

# Results

In [ ]:
nandf[['Multivariate LSTM','True data','OE-LSTM']].plot(kind='bar',figsize=(10,5),fontsize=10,color=['#ff7f0e','#1f77b4','#2ca02c'])

In [ ]:
#nandf[['diff','old_diff']].abs().plot(kind='bar',figsize=(20,10),title='Advanced traffic flow outlier predictions')

# Mapes

In [ ]:
mape_dat = {'Naive LSTM':[12.18,31.09],'OE-LSTM':[10.91,16.87]}

In [ ]:
mape_df = pd.DataFrame(mape_dat)

In [ ]:
mape_df.index = ['All test data','Top outliers']

In [ ]:
color_dict = {'Naive LSTM': '#1f77b4', 'OE-LSTM': '#2ca02c'}

# use get to specify dark gray as the default color.
mape_df.plot(kind='bar',title='MAPE comparison',color=['#ff7f0e','#2ca02c'])
plt.show()